In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
a2015003713_militaryaircraftdetectiondataset_path = kagglehub.dataset_download('a2015003713/militaryaircraftdetectiondataset')

print('Data source import complete.')

In [1]:
%pip install ultralytics --quiet

Note: you may need to restart the kernel to use updated packages.


In [1]:
from pathlib import Path
import pandas as pd
import yaml
import shutil
import torch
import cv2
from ultralytics import YOLO

In [3]:
dataset_dir = Path("data/military_dataset/versions/91/dataset")
output_dir = Path("yolo_dataset")
image_dir = output_dir / "images/train"
image_dir.mkdir(parents=True, exist_ok=True)
label_dir = output_dir /"labels/train"
label_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
all_classes = set()

for csv_file in dataset_dir.glob("*.csv"):
    try:
        df = pd.read_csv(csv_file)
        if "class" in df.columns:
            all_classes.update(df["class"].dropna().unique())
    except Exception as e:
        print(f"Error {csv_file.name}: {e}")

class_list = sorted(all_classes)

if not class_list:
    raise ValueError("No valid classes found in CSV files.")

data_yaml = {
    "path": str(output_dir.resolve()),
    "train": "images/train",
    "val": "images/train",
    "names": class_list
}

with open(output_dir / "data.yaml", "w") as f:
    yaml.dump(data_yaml, f, sort_keys=False)

In [4]:
for csv_file in dataset_dir.glob("*.csv"):
    name = csv_file.stem
    img = dataset_dir / f"{name}.jpg"
    if img.exists():
        shutil.copy(img, image_dir / f"{name}.jpg")

In [5]:
classes = sorted(list(set([
    row["class"]
    for csv in dataset_dir.glob("*.csv")
    for _, row in pd.read_csv(csv).iterrows()
    if "class" in row and pd.notna(row["class"])
])))

class_to_id = {cls: i for i, cls in enumerate(classes)}

for csv_file in dataset_dir.glob("*.csv"):
    name = csv_file.stem
    img_path = image_dir / f"{name}.jpg"
    if not img_path.exists():
        continue

    df = pd.read_csv(csv_file)
    if not {"xmin", "ymin", "xmax", "ymax", "class"}.issubset(df.columns):
        continue

    h, w = cv2.imread(str(img_path)).shape[:2]
    out = []
    for _, r in df.iterrows():
        if r["class"] not in class_to_id:
            continue
        xc = ((r["xmin"] + r["xmax"]) / 2) / w
        yc = ((r["ymin"] + r["ymax"]) / 2) / h
        bw = (r["xmax"] - r["xmin"]) / w
        bh = (r["ymax"] - r["ymin"]) / h
        out.append(f"{class_to_id[r['class']]} {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}")

    with open(label_dir / f"{name}.txt", "w") as f:
        f.write("\n".join(out))

In [6]:
for label_dir in sorted(label_dir.glob("*.txt"))[:10]:
    name = label_dir.stem
    image_file = image_dir / f"{name}.jpg"
    print(f"Checking: {label_dir.name}")

    if not image_file.exists():
        print("Image not found")
        continue

    lines = label_dir.read_text().strip().splitlines()
    if not lines:
        print("Empty label file")
        continue

    for line in lines:
        parts = line.split()
        if len(parts) != 5:
            print("Invalid line:", line)
            continue
        cls, xc, yc, bw, bh = parts
        print(f"Class: {cls}, xc: {xc}, yc: {yc}, bw: {bw}, bh: {bh}")

    print("Image found\n")

Checking: 0000e97ea2d086d6759b19b288a8a72c.txt
Class: 52, xc: 0.294440, yc: 0.600184, bw: 0.028815, bh: 0.033701
Class: 52, xc: 0.382204, yc: 0.518382, bw: 0.030235, bh: 0.041054
Class: 52, xc: 0.500406, yc: 0.505208, bw: 0.026786, bh: 0.047794
Class: 52, xc: 0.815239, yc: 0.561121, bw: 0.033482, bh: 0.036458
Image found

Checking: 00010041af654d0b8e1e16c824fa9867.txt
Class: 72, xc: 0.760294, yc: 0.309326, bw: 0.292647, bh: 0.104980
Image found

Checking: 000106393cfe2343888c584e65fd2274.txt
Class: 28, xc: 0.255114, yc: 0.288584, bw: 0.510227, bh: 0.577168
Image found

Checking: 0002f2cc10d90392efc3ad30cbda6490.txt
Class: 33, xc: 0.390456, yc: 0.667156, bw: 0.022673, bh: 0.053467
Class: 33, xc: 0.208542, yc: 0.548472, bw: 0.022673, bh: 0.053467
Class: 33, xc: 0.661745, yc: 0.373090, bw: 0.021619, bh: 0.045828
Class: 33, xc: 0.182310, yc: 0.533490, bw: 0.022937, bh: 0.052879
Class: 33, xc: 0.364355, yc: 0.649530, bw: 0.022673, bh: 0.054642
Class: 33, xc: 0.130108, yc: 0.497944, bw: 0.02

In [4]:
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")

CUDA available: True
GPU: NVIDIA GeForce RTX 2060


In [7]:
data_yaml_path = output_dir / "data.yaml"

model = YOLO("C:/Users/Andraf/runs/detect/train16/weights/best.pt")
model.train(data=str(data_yaml_path), epochs=30, imgsz=640, batch=16, cache=True)

New https://pypi.org/project/ultralytics/8.3.173 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.170  Python-3.12.0 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=yolo_dataset\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=C:/Users/Andraf/runs/detect/train16/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train1

train: Scanning C:\Users\Andraf\OneDrive - CY Cergy Paris Université\Machine Learning\DL\yolo_dataset\labels\train.cache... 20658 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20658/20658 [00:00<?, ?it/s]


WARNING train: 23.6GB RAM required to cache images with 50% safety margin but only 3.7/15.9GB available, not caching images
val: Fast image access  (ping: 0.30.1 ms, read: 81.347.8 MB/s, size: 126.3 KB)


val: Scanning C:\Users\Andraf\OneDrive - CY Cergy Paris Université\Machine Learning\DL\yolo_dataset\labels\train.cache... 20658 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20658/20658 [00:00<?, ?it/s]


WARNING val: 24.0GB RAM required to cache images with 50% safety margin but only 0.9/15.9GB available, not caching images
Plotting labels to c:\Users\Andraf\runs\detect\train17\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000112, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to c:\Users\Andraf\runs\detect\train17
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      30/30      4.72G     0.4413     0.4982     0.8804          2        640: 100%|██████████| 1292/1292 [05:48<00:00,  3.70it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 646/646 [02:11<00:00,  4.91it/s]


                   all      20658      36858      0.945      0.877      0.945      0.859

30 epochs completed in 3.783 hours.
Optimizer stripped from c:\Users\Andraf\runs\detect\train17\weights\last.pt, 22.6MB
Optimizer stripped from c:\Users\Andraf\runs\detect\train17\weights\best.pt, 22.6MB

Validating c:\Users\Andraf\runs\detect\train17\weights\best.pt...
Ultralytics 8.3.170  Python-3.12.0 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)
Model summary (fused): 72 layers, 11,158,479 parameters, 0 gradients, 28.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 646/646 [02:17<00:00,  4.71it/s]


                   all      20658      36858      0.944      0.878      0.945      0.859
                   A10        454        730      0.948       0.83      0.926      0.824
                 A400M        390        525      0.944      0.872      0.951      0.892
                 AG600        240        277      0.986      0.975      0.988       0.96
                  AH64        294        478      0.928      0.894      0.953       0.82
                  AV8B        282        450      0.937      0.863      0.936      0.876
                 An124        160        214      0.965      0.935      0.974       0.91
                  An22         96        104       0.98      0.936      0.965      0.914
                 An225         89        101      0.979      0.881      0.956      0.919
                  An72        148        183      0.969      0.869       0.93      0.865
                    B1        444        640      0.953      0.895       0.97      0.886
                    B

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
       78, 79, 80, 81, 82, 83, 84])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001E059ACC980>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0

In [ ]:
model = YOLO("YOLOv8s_trained/weights/best.pt")
output_image_path = Path("detection_example.jpg")

result = model.predict(
    source="127095687_2549772018654038_5724107342713412054_n.jpg",
    conf=0.25,
    save=False,
    show=False
)[0]

img = result.orig_img.copy()

for box in result.boxes:
    cls_id = int(box.cls)
    class_name = result.names[cls_id]
    conf = float(box.conf)
    x1, y1, x2, y2 = map(int, box.xyxy[0])

    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    label = f"{class_name} {conf:.2f}"
    (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
    cv2.rectangle(img, (x1, y1 - 20), (x1 + tw, y1), (0, 255, 0), -1)
    cv2.putText(img, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)

cv2.imwrite(str(output_image_path), img)
cv2.imshow("Detection", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


image 1/1 c:\Users\Andraf\OneDrive - CY Cergy Paris Universit\Machine Learning\DL\127095687_2549772018654038_5724107342713412054_n.jpg: 320x640 1 F16, 1 Su34, 10.1ms
Speed: 1.3ms preprocess, 10.1ms inference, 1.7ms postprocess per image at shape (1, 3, 320, 640)
